In [1]:
from scipy import stats
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS, utils
from photutils import SkyCircularAnnulus, SkyCircularAperture, aperture_photometry
from regions import CircleSkyRegion
import numpy as np

import glob
import os
import subprocess
import sys
import csv

In [2]:
sas_dir = "/home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155"
os.environ["SAS_DIR"] = sas_dir
os.environ["SAS_PATH"] = os.environ["SAS_DIR"]
os.environ["SAS_VERBOSITY"] = "4"
os.environ["SAS_SUPPRESS_WARNING"] = "1"
path = os.environ["PATH"]
os.environ["PATH"] = f"{sas_dir}/bin:{sas_dir}/binextra:{path}"
if "LD_LIBRARY_PATH" in os.environ.keys():
    ld_path = os.environ["LD_LIBRARY_PATH"]
# lib_path = f"{sas_dir}/lib:{sas_dir}/libextra:{sas_dir}/libsys:{ld_path}"
lib_path = f"{sas_dir}/lib:{sas_dir}/libextra:{sas_dir}"
os.environ["LD_LIBRARY_PATH"] = lib_path
os.environ["PERL5LIB"] = "{}/lib/perl5".format(sas_dir)
os.environ['SAS_CCFPATH'] = "/home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155/calibration"
!sasversion
!perl -e "print qq(@INC)"

sasversion:- Executing (routine): sasversion  -w 1 -V 4
sasversion:- sasversion (sasversion-1.3)  [xmmsas_20190531_1155-18.0.0] started:  2020-06-02T07:44:56.000
sasversion:- XMM-Newton SAS release and build information:

SAS release: xmmsas_20190531_1155-18.0.0
Compiled on: Sun Jun  2 14:11:18 CEST 2019
Compiled by: sasbuild@sasbld03n
Platform   : Ubuntu16.04 64

SAS-related environment variables that are set:

SAS_DIR = /home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155
SAS_PATH = /home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155
SAS_CCFPATH = /home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155/calibration

sasversion:- sasversion (sasversion-1.3)  [xmmsas_20190531_1155-18.0.0] ended:    2020-06-02T07:44:56.000
/home/aaranda/SAS/sas_18.0.0-Ubuntu16.04-64/xmmsas_20190531_1155/lib/perl5 /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.26.1 /usr/local/share/perl/5.26.1 /usr/lib/x86_64-linux-gnu/perl5/5.26 /usr/share/perl5 /usr/lib/x86_64-linux-g

In [3]:
def exec_task(task, verbose=True):
    try:
        # Write the shell output to tmp.log file.
        fout = open("tmp.log", "w")
        result = subprocess.run(task, shell=True, stdout=fout, stderr=subprocess.STDOUT)
        retcode = result.returncode
        fout.close()
        if retcode < 0:
            if (verbose):
                print(f"Execution of {task} was terminated by code {-retcode}.", file=sys.stderr)
        else:
            if (verbose):
                print(f"Execution of {task} returned {retcode}.", file=sys.stderr)
    except OSError as e:
        print(f"Execution of {task} failed:", e, file=sys.stderr)
    return retcode

In [4]:
def get_coords(odf_dir):
    logs_list = glob.glob("{}/logs/rad_prof*".format(odf_dir))
    aux = False
    aux_r = []
    circle = ""
    for log in logs_list:
        if not aux:
            with open(log, "r") as f:
                for line in f:
                    if "CIRCLE: " in line:
                        #circle = line.split(" ")[1]
                        circle = line.split("- ")[1].split(" ")[1]
                        #print(circle)
                if circle != "":
                    aux_r.append(float(circle.split(',')[2]))
                    #aux = True
    RA = float(circle.split(',')[0])
    DEC = float(circle.split(',')[1])
    r = max(aux_r)
    print(RA, DEC, r)
    return [RA, DEC, r]

In [5]:
def calc_radial_profile(fitsfile, center, rstart, rend, rstep, verbose=False, detmaskfile=None, plot=True):
    """

    Utility function to calculate the radial profile from an image `fitsfile` at a `center`

    """
    #
    if (not os.path.isfile(fitsfile)):
        print(f"ERROR. FITS file {fitsfile} not found. Cannot continue.")
        return None
    #
    qhdu = fits.open(fitsfile)
    wcs = WCS(qhdu[0].header)
    #
    # if detmaskfile is provided then will use it for detector mask
    #
    doMask = False
    if (detmaskfile != None):
        if (not os.path.isfile(detmaskfile)):
            print(f"Warning. Detector mask file {detmaskfile} not found. Will not use detector mask!")
            doMask = False
        else:
            det = fits.open(detmaskfile)
            detmask = det['MASK']
            # need the WCS
            wcs_det = WCS(detmask.header)
            doMask = True
    #
    if (not isinstance(center, SkyCoord)):
        print(f"EROOR: the input radial profile centre is not SkyCoord object. Cannot continue.")
        return None
    #
    j = 0
    rx = rstart
    counts = []
    counts_err = []
    rmid = []
    #
    emtpy = False
    while rx < r_end:
        r0 = rstart + rstep * j
        rx = rstart + rstep * (j + 1)
        # the mid point, can be better the mid area point
        rmid.append((r0.value + rx.value) / 2.0)
        if (j == 0):
            xap = SkyCircularAperture(center, rx)
            photo = aperture_photometry(qhdu[0].data, xap, wcs=wcs)
            if (doMask):
                masked = aperture_photometry(detmask.data, xap, wcs=wcs_det)
        else:
            xap = SkyCircularAnnulus(center, r0, rx)
            photo = aperture_photometry(qhdu[0].data, xap, wcs=wcs)
            if (doMask):
                masked = aperture_photometry(detmask.data, xap, wcs=wcs_det)
        #
        ap_area = xap.to_pixel(wcs).area
        good_area = ap_area
        if (doMask):
            good_area = masked['aperture_sum'][0]
        # compare the two annuli areas: with and without bad pixels
        if (verbose):
            print(
                f"Annulus: {r0:.2f},{rx:.2f},geometric area: {ap_area:.1f} pixels,non-masked area {good_area:.1f} pixels, ratio: {ap_area / good_area:.2f}")
        # taking into account the masked pixels
        if (good_area == 0.0):
            counts.append(float('nan'))
            counts_err.append(float('nan'))
        else:
            counts.append(photo['aperture_sum'][0] / good_area)
            counts_err.append(np.sqrt(photo['aperture_sum'][0]) / good_area)

        j += 1
    #
    # convert the results in numpy arrays
    #
    rmid = np.array(rmid)
    counts = np.array(counts)
    counts_err = np.array(counts_err)
    #
    # convert per pixel to per arcsec^2
    pix_area = utils.proj_plane_pixel_area(wcs) * 3600.0 * 3600.0  # in arcsec^2
    counts = counts / pix_area
    counts_err = counts_err / pix_area
    

    return rmid, counts, counts_err

In [6]:
def psf_gen(center, energy, box, psf_out):
    psfgen = f'psfgen image={fits_image} withimage=yes instrument=PN level=ELLBETA energy={energy} ' + \
             f'x={center.ra.value} y={center.dec.value} coordtype=EQPOS xsize={box} ysize={box} output={psf_out}'
    status = exec_task(psfgen)
    if (status != 0):
        raise RuntimeError

In [7]:
# Good obsid_lits extracter from find_valid_targets.py
obsid_list = ['0142270101', '0300480201', '0300480301', '0401060201', '0502630201', '0679380701', '0761500201',
              '0651360401', '0800732801', '0762870401', '0122520201', '0674330201', '0653040101', '0744412401',
              '0783881001', '0302580401', '0502430201', '0502430701', '0693010401', '0106060601', '0112550801',
              '0112551101', '0721890101', '0148250201', '0300140101', '0149880101', '0303930101', '0650380201',
              '0693741001', '0153170101', '0084140101', '0084140501', '0655343842', '0002970201', '0604210201',
              '0604210301', '0151490101', '0785110401', '0152900201', '0740820401', '0123100101', '0123100201',
              '0670040101', '0761112401', '0803240801', '0830190501', '0303820301', '0783520201', '0652810201',
              '0152460301', '0305750601', '0551750301', '0600920201', '0112830201', '0112830501', '0104860501',
              '0124900101', '0692510101', '0604830201', '0100240101', '0100240201', '0804272701', '0722140101',
              '0722140401', '0800270601', '0790380501', '0790380601', '0790380801', '0790381401', '0790381501',
              '0790380901', '0761100101', '0761100201', '0761100301', '0761100401', '0761101001', '0744640101',
              '0553561101', '0655346840', '0673000136', '0405380701', '0782350501']


wdir = "/home/aaranda/tfm/obsid"

bll_catalog = fits.open('/home/aaranda/tfm/bllacs_PN_NopileupNofasttiming.fits')
data = bll_catalog[1].data

bin_size = 80
pi = [500, 2000, 4500, 10000]

error_obsid = []
count = 1
# obsid_list = ['0830190501']
for obsid in obsid_list:

    try:

        # Find target name
        for row in data:
            if row[5] == obsid:
                target = row[0]

        print("Executing {} of {}...".format(count, len(obsid_list)))
        count += 1
        odf_dir = "{}/{}".format(wdir, obsid)

        os.chdir(odf_dir)
        cif_file = "{}/ccf.cif".format(odf_dir)
        os.environ['SAS_CCF'] = cif_file
        SUM_SAS_file = glob.glob("*SUM.SAS")[0]
        os.environ['SAS_ODF'] = SUM_SAS_file

        coords = get_coords(odf_dir)
        center = SkyCoord(coords[0], coords[1], unit=(u.deg, u.deg), frame='icrs')
        r_start = 0.0 * u.arcsec
        r_end = 4.0 * u.arcmin
        r_step = 6.0 * u.arcsec

        detmask_file = '{}/detfile_low.fits'.format(odf_dir)
        det = fits.open(detmask_file)
        detmask = det['MASK']
        wcs_det = WCS(detmask.header)

        fits_image = '{}/images/image_filtered_low.fits'.format(odf_dir)



        (x, y, yerr) = calc_radial_profile(fits_image, center, r_start, r_end, r_step, verbose=False,
                                           detmaskfile=detmask_file, plot=False)

        
        
        energy = 1000  # at 1 keV
        box = 2 * int(r_end.to(u.arcsec).value) + 1
        psf_out = '{}/psf_ellbeta_{}_{}pix.fits'.format(odf_dir, energy, box)
        psf_gen(center, energy, box, psf_out)

        (psf_mid, psf_counts, psf_counts_err) = \
            calc_radial_profile(psf_out, center, r_start, r_end, r_step, verbose=False, detmaskfile=None, plot=False)

        #
        # normalise the source
        #
        jnorm = next((i for i, yy in enumerate(y) if (yy > 0.0)), None)
        #
        if (jnorm > 0):
            print("Warning: the central radial bin is masked (zero area) or has zero counts.")
            print(
                f"The first non-zero radial bin is with index {jnorm}, bin start at {r_start + r_step * jnorm} arcsec")
        norm_counts = y / y[jnorm]
        norm_counts_err = yerr / y[jnorm]
        back = np.mean(norm_counts[-3:])  # the mean of the last 3 radial points
        #
        # and normalise the PSF
        #
        norm_psf_counts = psf_counts / psf_counts[jnorm]
        #
        
        result = stats.ks_2samp(norm_psf_counts + back, norm_counts)
        print(result[0], result[1])
        
        
        with open('/home/aaranda/tfm/kolmorogov_low.csv', mode='a') as csv_file:
            csv_file = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_file.writerow([obsid, result[0], result[1]])
        
    except:
        error_obsid.append(obsid)
        print("Error ocurred with obsid: {}".format(obsid))
        pass

print(error_obsid)

Executing 1 of 82...
87.6688 -32.2711 0.03333333333333333


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Execution of psfgen image=/home/aaranda/tfm/obsid/0142270101/images/image_filtered_low.fits withimage=yes instrument=PN level=ELLBETA energy=1000 x=87.6688 y=-32.2711 coordtype=EQPOS xsize=481 ysize=481 output=/home/aaranda/tfm/obsid/0142270101/psf_ellbeta_1000_481pix.fits returned 0.


0.175 0.5786001416508443
Executing 2 of 82...
133.7033 20.1083 0.03333333333333333


Execution of psfgen image=/home/aaranda/tfm/obsid/0300480201/images/image_filtered_low.fits withimage=yes instrument=PN level=ELLBETA energy=1000 x=133.7033 y=20.1083 coordtype=EQPOS xsize=481 ysize=481 output=/home/aaranda/tfm/obsid/0300480201/psf_ellbeta_1000_481pix.fits returned 0.


0.25 0.16497269950224194
Executing 3 of 82...
133.7033 20.1083 0.03333333333333333


...

0.5 6.5768913245274e-05
Executing 81 of 82...
349.7742 -42.1136 0.03333333333333333


Execution of psfgen image=/home/aaranda/tfm/obsid/0405380701/images/image_filtered_low.fits withimage=yes instrument=PN level=ELLBETA energy=1000 x=349.7742 y=-42.1136 coordtype=EQPOS xsize=481 ysize=481 output=/home/aaranda/tfm/obsid/0405380701/psf_ellbeta_1000_481pix.fits returned 0.


0.175 0.5786001416508443
Executing 82 of 82...
349.9308 16.1975 0.03333333333333333


Execution of psfgen image=/home/aaranda/tfm/obsid/0782350501/images/image_filtered_low.fits withimage=yes instrument=PN level=ELLBETA energy=1000 x=349.9308 y=16.1975 coordtype=EQPOS xsize=481 ysize=481 output=/home/aaranda/tfm/obsid/0782350501/psf_ellbeta_1000_481pix.fits returned 0.


0.325 0.02860307028023343
[]
